<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NBXX__SeriesTemporaisUnivariadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Referências
* [Time Series Forecasting With Prophet in Python](https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/);
* [Time Series in Driverless AI](http://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/time-series.html) --> Pegar um exemplo daqui!!!

## Relato do uso de AutoML nas competições do Kaggle
https://ai.googleblog.com/2020/12/using-automl-for-time-series-forecasting.html?m=1

## Outras alternativas
* H2O - http://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/time-series.html
* auto_ts

## Prophet
* Prophet ou "Facebook Prophet" é uma library desenvolvida pelo Facebook para lidar com séries temporais univariadas (1 única variável).
* Suporte para:
    * Tendência;
    * Sazonalidade;
    * Periodicidade;
    * Estacionariedade;

## Instalar a library Prophet

In [ ]:
!pip install fbprophet

## Carregar a library Prophet

In [ ]:
import fbprophet
print(f'versão da library FBProphet: {fbprophet.__version__}')

## Carregar as libraries necessárias

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

## Exemplo 1

### Carregar os dados para análise de Séries Temporais

In [ ]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = pd.read_csv(path, heade = 0)
df.head()

In [ ]:
print(df.shape)

### Gráfico da série temporal

In [ ]:
df.plot()
plt.show()

### Características da série temporal
* Percebe-se claramente que a série temporal apresenta **tendência** e **sazonalidade**.

* Como podemos ver a **tendência** e a **sazonalidade**?

### Como usar a library Prophet
* Os dados (o dataframe) precisam estar num formato definido:
    * A 1ª coluna do dataframe tem que ter o nome **ds** e possuir as datas/tempos da série temporal;
    * A 2 coluna do dataframe tem que ter o nome **y** e deve conter as observações da série temporal.

> Portanto, precisamos aplicar algumas transformações ao nosso dataframe para obedecer estes requisitos.

In [ ]:
df.dtypes

As transformações a serem feitas são:
* A coluna **Month** será renomeada para **ds** (para atender os requisitos da library Prophe) e, na sequência, a transformaremos para o tipo datetime, pois ela originalmente é do tipo object;
* A coluna **Sales** será renomeada para **y**.

In [ ]:
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds']) # Transformando ds para datetime.

In [ ]:
df.head()

In [ ]:
df.dtypes

### Definição do modelo para a série temporal

In [ ]:
from fbprophet import Prophet
st_ml = Prophet() # instanciamento do objeto

#### Ajuste do modelo de série temporal

In [ ]:
df.head()

In [ ]:
st_ml.fit(df)

### Validação do modelo de série temporal

In [ ]:
df.tail()

In [ ]:
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
	date = '1968-%02d' % i
	future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= pd.to_datetime(future['ds'])

In [ ]:
# use the model to make a forecast
forecast = st_ml.predict(future)
#forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
st_ml.plot(forecast)
plt.show()

### Forecast com a série temporal modelada

In [ ]:
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
	date = '1969-%02d' % i
	future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= pd.to_datetime(future['ds'])

In [ ]:
forecast2 = st_ml.predict(future)
print(forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
# plot forecast
st_ml.plot(forecast2)
plt.show()

### Validação com MAE

In [ ]:
# calculate MAE between expected and predicted values for december
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
y_true = df['y'][-12:].values
y_pred = forecast['yhat'].values
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
print('MAE: %.3f' % mae)
print('MSE: %.3f' % mse)

In [ ]:
# plot expected vs actual
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

## Exemplo 2

In [ ]:
url = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/multiTimeline.csv'
df = pd.read_csv(url)
df.head()

#### Renomear o nome das colunas
* A coluna

In [ ]:
df.columns = ['ds', 'diet', 'gym', 'finance']
df.head()

#### Transformar a coluna **ds** para o tipo datetime e que apontá-la como sendo o índice da série temporal

In [ ]:
df.ds = pd.to_datetime(df.ds) # transforma ds num objeto do tipo datetime.
df.set_index('ds', inplace = True)
df.head()

### Gráficos - EDA (Análise Exploratória de Dados)
* na figura abaixo, o gráfico das três séries temporais juntas no mesmo gráfico.

In [ ]:
df.plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Year', fontsize = 20);

* A seguir, o gráfico somente da série temporal **diet**.
    * Quais os padrões (patterns) que vocês conseguem identificar nesse gráfico?
        * Padrões presentes na série temporal: tendência (trends) e componentes sazonais.

In [ ]:
df[['diet']].plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Year', fontsize = 20)

### Análise da Tendência
* Há várias formas para se avaliar tendência nas séries temporais. Talvez a maneira mais popular seja fazer médias móveis.
    * Médias Móveis significa que, para cada ponto no tempo, você tira a média dos pontos de cada lado dele (o número de pontos é especificado por um tamanho de janela, que você precisa escolher).

A seguir, função para nos ajudar com o DataViz:

In [ ]:
def grafico_ts(df, coluna):
    df2 = df[[coluna]]
    
    df3 = pd.concat([df2.rolling(24).mean(), 
                     df2.rolling(18).mean(), 
                     df2.rolling(12).mean(), 
                     df2.rolling(6).mean(), 
                     df2.rolling(3).mean()
                     ], axis=1)

    df3.plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
    plt.xlabel('Year', fontsize = 20)

A seguir, o gráfico de médias para a série temporal **diet**:

In [ ]:
grafico_ts(df, 'diet')

A seguir, o gráfico de médias móveis para a série temporal **gym**:

In [ ]:
grafico_ts(df, 'gym')

A seguir, função para nos ajudar com o DataViz nos casos em que as séries temporais são distintas:

In [ ]:
def grafico_ts_distintas(df, coluna1, coluna2, mm):
    df1 = df[[coluna1]]
    df2 = df[[coluna2]]

    df3 = pd.concat([df1.rolling(mm).mean(), 
                     df2.rolling(mm).mean()
                     ], axis=1)

    df3.plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
    plt.xlabel('Year', fontsize = 20)

A seguir, o gráfico conjunto das séries temporais **diet** e **gym**:

In [ ]:
df.head()

In [ ]:
grafico_ts_distintas(df, 'diet', 'gym', 6)

In [ ]:
grafico_ts_distintas(df, 'diet', 'gym', 6)

### Padrões/Componentes sazonais
* Podemos remover a tendência de uma série temporal para investigar a sazonalidade;
* Para remover a tendência, você pode subtrair a tendência calculada acima (média móvel) do sinal original. Isso, no entanto, dependerá de quantos pontos de dados você calculou.
* Outra maneira de remover a tendência é chamada de **diferenciação**.

#### Diferenciação de 1ª Ordem

In [ ]:
df_diet = df[['diet']].copy()
df_diet_diff = df[['diet']].copy()

df4 = pd.concat([df_diet, df_diet_diff.diff()], axis = 1)

df4.plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Year', fontsize = 20)

**Comentários:** Observe que foi removida grande parte da tendência da série temporal original (em azul). Desta forma, podemos observar claramente os picos em janeiro de cada ano. 
* Caso a tendência ainda não tenha sido removida, podemos usar a diferenciação de 2ª ordem. Entretanto, na prática, as diferenciações de 1ª e 2ª ordem são suficientes para remover a tendência das séries temporais.
* **Séries temporais estacionárias**: O operador **diferença** é muito útil para transformar a série temporal original em uma série temporal **estacionária**.
    * **Séries temporias estacionárias possuem propriedades estatísticas como média e variância que NÃO VARIAM COM O TEMPO**. 
    * **As séries temporais estacionárias são úteis porque muitos métodos de previsão de séries temporais são baseados na suposição de que a série temporal é aproximadamente estacionária**.

### Periodicidade e autocorrelação
* Uma série temporal é periódica se ela se repetir em intervalos igualmente espaçados, digamos, a cada 12 meses. 
* Outra maneira de se observar a periodicidade é avaliar os picos em épocas específicas da série. Por exemplo, no Natal podemos observar picos de vendas.
* Outra forma é avaliar a autocorrelação serial. Ou seja, avaliar a correlação da série temporal com ela mesma no passado.

#### Correlação
* O coeficiente de correlação de duas variáveis captura o quão linearmente relacionadas estão as duas variáveis

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
df_iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
df_iris.head()

In [ ]:
sns.lmplot(x='sepal length (cm)', y='sepal width (cm)', fit_reg=False, data=df_iris);

In [ ]:
sns.lmplot(x='sepal length (cm)', y='sepal width (cm)', fit_reg=False, data=df_iris, hue='target');

**Pergunta**: **sepal length** e **width** estão positiva ou negativamente correlacionados?

* Vamos calcular o coeficiente de correlação:

In [ ]:
df_iris.corr()

Observe que 'sepal length (cm)' e 'sepal width (cm)' parecem estar negativamente correlacionados!

### Análise de Correlação de Séries Temporais

In [ ]:
df.plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Year', fontsize = 20);

A seguir, o Coeficiente de Correlação entre as séries temporais:

In [ ]:
df.corr()

A seguir, o gráfico usando as diferenças de 1ª ordem dessas séries temporais (a remoção da tendência pode revelar correlação na sazonalidade):

In [ ]:
df.diff().plot(figsize = (20, 10), linewidth = 5, fontsize = 20)
plt.xlabel('Year', fontsize = 20);

A seguir, os Coeficientes de Correlação das diferenças de 1ª Ordem dessas séries temporais (a remoção da tendência pode revelar correlação na sazonalidade):

In [ ]:
df.diff().corr()

### Autocorrelação
* Entendemos como funciona a correlação entre variáveis e séries temporais. Agora vamos entender e plotar os gráficos de autocorrelação da série temporal **dieta**.
    * Se a série temporal original se repete a cada dois dias, você esperaria ver um pico na função de autocorrelação em 2 dias.

In [ ]:
pd.plotting.autocorrelation_plot(diet)

## Modelos ARIMA (**A**uto**R**egressive **I**ntegrated **M**oving **A**verage Model)
* Modelos estatísticos clássicos para analisar e prever dados de séries temporais;
* Resumidamente, temos:
    * AR: **Autorregressão** - Modelo que usa a relação dependente entre uma observação e algum número de observações defasadas;
    * I: **Integrado** - Diz respeito às diferenciações necessárias para tornar a série temporal **estacionária**;
    * MA: **Média móvel** - Modelo que usa a dependência entre uma observação e um erro residual de um modelo de média móvel aplicado a observações defasadas.

Os parâmetros do modelo ARIMA são definidos da seguinte forma:

* **p**: número de observações de defasagem incluídas no modelo;
* **d**: número de diferenciações para tornar a série temporal estacionária;
* **q**: O tamanho da janela da média móvel, também chamada de ordem da média móvel.

In [ ]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA, ARIMA
import itertools

# ARCH, GARCH, TGARCH e etc
# LSTM --> Redes Neurais

In [ ]:
df.head()

In [ ]:
# Grid Search
p = d = q = range(0, 5) # p, d, e q podem ser 0, 1, 2, 3 ou 4
pdq = list(itertools.product(p, d, q)) # Todas as combinações possiveis de p, d, and q

In [ ]:
X_treinamento, X_teste = df.iloc[:162, :].copy(), df.iloc[162:, :]
X_treinamento.shape, X_teste.shape

In [ ]:
df.tail()

### Modelo de Séries Temporais usando Prophet

In [ ]:
X_treinamento = X_treinamento.drop(columns = ['gym', 'finance'], axis = 1)
X_teste = X_teste.drop(columns = ['gym', 'finance'], axis = 1)

In [ ]:
X_treinamento = X_treinamento.rename({'diet': 'y'}, axis = 1)
X_teste.rename({'diet': 'y'}, axis = 1)

In [ ]:
ts = Prophet()

In [ ]:
ts.fit(X_treinamento)